In [1]:
# USE THE LARGE ONE!!!
import os
from huggingface_hub import login
token = os.getenv("GITHUB_TOKEN")
login(token=token)

In [2]:
import os, yaml, sys
import torch
from einops import reduce, rearrange
from transformers import pipeline
from transformers import logging
logging.set_verbosity_error()
from huggingface_hub import login
token = os.getenv("GITHUB_TOKEN")
login(token=token)
pipe = pipeline(
    task="image-feature-extraction",
    model="facebook/dinov3-vits16-pretrain-lvd1689m",
    dtype=torch.bfloat16,
)

f = pipe("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/pipeline-cat-chonk.jpeg")

W0103 19:31:48.821000 39273 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


In [3]:
ENV = os.getenv("MY_ENV", "dev")
with open("../../config.yaml", "r") as f:
    config = yaml.safe_load(f)
paths = config[ENV]["paths"]
sys.path.append(paths["src_path"])
from general_utils.utils import get_device, get_module_by_path
from image_processing.utils import get_activation
device = get_device()


19:31:57 - device being used: mps


In [4]:
import torch
from transformers import AutoImageProcessor, AutoModel
from transformers.image_utils import load_image
import accelerate
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = load_image(url)

processor = AutoImageProcessor.from_pretrained("facebook/dinov3-vits16-pretrain-lvd1689m")
model = AutoModel.from_pretrained(
    "facebook/dinov3-vits16-pretrain-lvd1689m",
    dtype=torch.float16,
    attn_implementation="sdpa"
).to(device)

inputs = processor(images=image, return_tensors="pt").to(model.device)
with torch.inference_mode():
    outputs = model(**inputs)

pooled_output = outputs.pooler_output
print("Pooled output shape:", pooled_output.shape)

Pooled output shape: torch.Size([1, 384])


In [5]:
# Dictionary to store intermediate features
features = {}
layer_name = "layer.0.mlp.down_proj"
module = get_module_by_path(model, layer_name)
module._forward_hooks.clear()
handle = module.register_forward_hook(get_activation(layer_name, features, 'all'))

# Run forward pass
with torch.no_grad():
    outputs = model(**inputs)
# Access features
print(features[layer_name].shape)

torch.Size([1, 77184])


In [ ]:
# m = model.layer[0].mlp.down_proj
# m._forward_hooks.clear()
# print(m._forward_hooks)

OrderedDict()


In [28]:
dir(model)

['T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_adjust_missing_and_unexpected_keys',
 '_apply',
 '_auto_class',
 '_backward_compatibility_gradient_checkpointing',
 '_backward_hooks',
 '_backward_pre_hooks',
 '_buffers',
 '_call_impl',
 '_can_compile_fullgraph',
 '_can_record_outputs',
 '_can_set_attn_implementation',
 '_check_and_adjust_attn_implementation',
 '_checkpoint_conversion_mapping',
 '_compiled_call_impl',
 '_copy_lm_head_original_to_resized',
 '_create_repo',
 '_dispatch_accelerate_model',
 '_ep_plan',
 '_fix_state_dict_key_on_load',
 '_fix_state_dict_key_o

In [5]:
# Print model modules and parameter shapes (uses existing `model` and `torch` from the notebook)
def print_model_layer_shapes(model):
    total_params = 0
    trainable_params = 0

    for module_name, module in model.named_modules():
        # skip the top-level empty name unless it has no parent info to show
        params = list(module.named_parameters(recurse=False))
        if not params:
            continue

        print(f"Module: '{module_name}' ({module.__class__.__name__})")
        for p_name, p in params:
            shape = tuple(p.shape)
            numel = p.numel()
            total_params += numel
            if p.requires_grad:
                trainable_params += numel
            print(f"  param: {p_name:20s} shape={str(shape):20s} numel={numel:10d} dtype={p.dtype}")
        print()

    print(f"Total params: {total_params:,}")
    print(f"Trainable params: {trainable_params:,}")

# Run the printer
print_model_layer_shapes(model)

Module: 'embeddings' (DINOv3ViTEmbeddings)
  param: cls_token            shape=(1, 1, 384)          numel=       384 dtype=torch.float16
  param: mask_token           shape=(1, 1, 384)          numel=       384 dtype=torch.float16
  param: register_tokens      shape=(1, 4, 384)          numel=      1536 dtype=torch.float16

Module: 'embeddings.patch_embeddings' (Conv2d)
  param: weight               shape=(384, 3, 16, 16)     numel=    294912 dtype=torch.float16
  param: bias                 shape=(384,)               numel=       384 dtype=torch.float16

Module: 'layer.0.norm1' (LayerNorm)
  param: weight               shape=(384,)               numel=       384 dtype=torch.float16
  param: bias                 shape=(384,)               numel=       384 dtype=torch.float16

Module: 'layer.0.attention.k_proj' (Linear)
  param: weight               shape=(384, 384)           numel=    147456 dtype=torch.float16

Module: 'layer.0.attention.v_proj' (Linear)
  param: weight               

In [9]:
# Dictionary to store features
features = {}

def hook_fn(module, input, output):
    features["layer0_scale1"] = output.detach()

# Register hook
target_layer = model.layer[0].layer_scale1
hook_handle = target_layer.register_forward_hook(hook_fn)

In [13]:
print(inputs['pixel_values'].shape)

torch.Size([1, 3, 224, 224])


In [ ]:
inputs = processor(images=image, return_tensors="pt").to(model.device)

with torch.inference_mode():    
    outputs = model(**inputs)

# Extract the saved feature
feat = features["layer0_scale1"]
print("Hooked feature shape:", feat.shape)

Hooked feature shape: torch.Size([1, 201, 384])
